In [ ]:
# Centealized dataset unification
import pandas as pd
import xgboost as xgb
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTENC
import matplotlib.pyplot as plt
from PIL import Image
from scipy.interpolate import BSpline, make_interp_spline, interp1d
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import csv
from dfply import *
from xgboost import XGBClassifier
import itertools
import os
import logging
from sys import getsizeof

In [ ]:
%load_ext magics

In [ ]:
#%%exceptor

site='UTHSCSA'
sep=','
ext='csv'
datafolder="/home/hchan2/AKI/data2022/"+site+"/raw/"
datafolderold="/home/hchan2/AKI/data/"+site+"/raw/"

pd.set_option('display.max_rows', 100)

In [ ]:
#%%exceptor

filename = 'CONSORT_DIAGRAM_BC'
consort_diagram = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
#%%exceptor

filenameX = 'CONSORT_DIAGRAM'
extX = 'csv'
#filenameX = 'Consort_Diagram'
consort_diagramX = pd.read_csv(datafolderold+filenameX+'.'+extX,sep=',')

In [ ]:
#%%exceptor

consort_diagram.merge(consort_diagramX, left_on = 'CNT_TYPE', right_on = 'CNT_TYPE', how='left')

In [ ]:
#%%exceptor

filename = 'AKI_ONSETS'
onset = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-8')

In [ ]:
onset[['ENCOUNTERID']].drop_duplicates().count()

In [ ]:
#%%exceptor

filename = 'AKI_VITAL'
vital = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
#%%exceptor

filename = 'DEID_AKI_VITAL_OLD'
vital_old = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-8')

In [ ]:
%%exceptor

filename = 'AKI_AMED'
amed = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
%%exceptor

filename = 'AKI_PMED'
pmed = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
%%exceptor

filename = 'AKI_DX'
dx = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
#%%exceptor

filename = 'DEID_AKI_DX_CURRENT_ADMIT_DATE'
dx_curr = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-8')

In [ ]:
%%exceptor

filename = 'AKI_PX'
px = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
#%%exceptor

filename = 'AKI_DEMO'
demo = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, na_values=['NI'], encoding = 'utf-8')

In [ ]:
%%exceptor

filename = 'AKI_DEMO_DEATH'
demod = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
%%exceptor

filename = 'AKI_DMED'
dmed = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
%%exceptor

filename = 'AKI_LAB_SCR'
lab_scr = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')

In [ ]:
#%%exceptor

filename = 'AKI_LAB'
lab = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, na_values=['NI'], encoding='utf-16', quoting=csv.QUOTE_NONE)

In [ ]:
consort_diagramX
# filename = 'CONSORT_DIAGRAM_BC'
# #filename = 'Consort_Diagram_BC'
# #filename = 'consort_diagram_BC'
# consort_diagram = pd.read_csv(datafolder+filename+'.'+ext,sep=sep, encoding = 'utf-16')
# consort_diagram.columns = ['CNT_TYPE', 'ENC_CNT_NEW']
# consort_diagram.merge(consort_diagramX, left_on='CNT_TYPE', right_on='CNT_TYPE', how='left')

In [ ]:
#%%exceptor

def countna(datacol): return datacol.isna().sum()
def countnotna(datacol): return datacol.notna().sum()
def float_data_stat(datatt):
    return datatt.select_dtypes(['float64']).agg(['count', np.mean, np.var, np.std, np.median,np.max, np.min, countna, countnotna]).T
def int_data_stat(datatt):
    return pd.melt(datatt.select_dtypes(['int64']).drop([x if 'ID' in x else 'dummy' for x in datatt.columns],axis=1,errors='ignore')).groupby(['variable','value'],dropna=False).size()
#    return pd.melt(datatt.select_dtypes(['int64'])).groupby(['variable','value'],dropna=False).size()
def obj_data_stat(datatt):
    return pd.melt(datatt.select_dtypes(['object'])).groupby(['variable','value'],dropna=False).size()
def countna_data_set(datatt):
    return datatt.agg([countna, countnotna]).T

In [ ]:
#%%exceptor

float_data_stat(onset)

In [ ]:
#%%exceptor

int_data_stat(onset)

In [ ]:
#%%exceptor

float_data_stat(vital_old)

In [ ]:
#%%exceptor

#int_data_stat(vital)
#np.logical_and(vital['DAYS_SINCE_ADMIT'] < 7, vital['DAYS_SINCE_ADMIT'] > 0).sum()
#(vital['DAYS_SINCE_ADMIT']==7).sum()
#plt.hist(vital['DAYS_SINCE_ADMIT'].quantile(0.995))
#plt.show() 
#int_data_stat(vital)
#vital['DAYS_SINCE_ADMIT']
#dfx = pd.merge(vital[['ONSETS_ENCOUNTERID']], onset[['ENCOUNTERID']], left_on='ONSETS_ENCOUNTERID', right_on='ENCOUNTERID', how='outer')
#countna(dfx)
#dataY = vital_old
dataY[dataY['DAYS_SINCE_ADMIT']>=0][dataY['DAYS_SINCE_ADMIT']<=30]['DAYS_SINCE_ADMIT'].hist()
#onset['ENCOUNTERID'].drop_duplicates().count()

In [ ]:
#%%exceptor

#obj_data_stat(vital)

In [ ]:
amed.isna().sum()/amed.shape[0]

In [ ]:
pmed.isna().sum()/pmed.shape[0]

In [ ]:
dmed.isna().sum()/pmed.shape[0]

In [ ]:
dx.isna().sum()/dx.shape[0]

In [ ]:
lab.isna().sum()/lab.shape[0]

In [ ]:
lab_scr.isna().sum()/lab_scr.shape[0]

In [ ]:
dx_curr.isna().sum()/dx_curr.shape[0]
#dx_curr.head(10)

In [ ]:
px.isna().sum()/px.shape[0]

In [ ]:
demod.isna().sum()/demod.shape[0]

In [ ]:
dmed.isna().sum()/dmed.shape[0]

In [ ]:
#%%exceptor

datatt[['DISPENSE_SOURCE']].groupby(['DISPENSE_SOURCE']).size()

In [ ]:
#%%exceptor

def group_count(datatt, dataname):
    print(dataname)
    for colname in datatt.columns:
        if 'ID' not in colname and 'RAW' not in colname:
            if datatt[[colname]].drop_duplicates().shape[0] <= 5:
                print(colname)
                print(datatt[[colname]].groupby(colname).size())
    print('---------------------------------------------------')
group_count(demo, 'demo')
group_count(vital, 'vital')
group_count(lab, 'lab')
group_count(lab_scr, 'lab_scr')
group_count(dmed, 'dmed')
group_count(amed, 'amed')
group_count(pmed, 'pmed')
group_count(px, 'px')
group_count(dx, 'dx')
group_count(dx_curr, 'dx')
group_count(demod, 'demod')

In [ ]:
#%%exceptor

group_count(onset, 'onset')

In [ ]:
#%%exceptor

def since_range(datatt, dataname):
    print(dataname)
    for colname in datatt.columns:
        if 'SINCE' in colname:
            print(datatt[[colname]].agg([np.max, np.min]).T)
    print('---------------------------------------------------')

In [ ]:
#%%exceptor

since_range(onset, 'onset')
since_range(demo, 'demo')
since_range(vital, 'vital')
since_range(lab, 'lab')
since_range(lab_scr, 'lab_scr')
since_range(dmed, 'dmed')
since_range(amed, 'amed')
since_range(pmed, 'pmed')
since_range(px, 'px')
since_range(dx, 'dx')
since_range(dx_curr, 'dx_curr')
since_range(demod, 'demod')

In [ ]:
#%%exceptor

def all_hist(datatt, dataname):
    print(dataname)
    for colname in datatt.columns:
        if 'ID' not in colname and 'RAW' not in colname:
            try:
                q = [0.025, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.975]
                dump, bins = pd.qcut(datatt[colname], q, retbins=True)
                datatt[[colname]].hist(bins=bins)
#                datatt[[colname]].hist()
                plt.show()
            except:
                try:
                    datatt[[colname]].hist()                    
                except:
                    if datatt[[colname]].drop_duplicates().shape[0] <= 100:
                        datatt[[colname]].value_counts().plot(kind='bar')
                        plt.show()                
                    else:
                        print('Too many columns for '+colname)
    print('----------------------------------')

In [ ]:
#%%exceptor

all_hist(onset, 'onset')
all_hist(demo, 'demo')
all_hist(vital, 'vital')
all_hist(lab, 'lab')
all_hist(lab_scr, 'lab_scr')
all_hist(dmed, 'dmed')
all_hist(amed, 'amed')
all_hist(pmed, 'pmed')
all_hist(px, 'px')
all_hist(dx, 'dx')
all_hist(dx_curr, 'dx_curr')
all_hist(demod, 'demod')